Exécution du TP1 afin d'avoir les phrases et les tokens du corpus Chirac

In [1]:
import math
%run TP1.ipynb


---------- Corpus Obama ------------

Le nombre de phrases est : 84
Le nombre de tokens est : 658
---------- Corpus Chirac ------------

Le nombre de phrases est : 37
Le nombre de tokens est : 293


In [2]:
import re

In [4]:
def split_into_sentences(text):
    return re.split(r'(?<=[.!?])\s+', text.strip())

In [5]:
def preprocess_sentences(sentences):
    unique_tokens = set()
    for sentence in sentences:
        tokens = re.findall(r'\b\w+\b', sentence.lower())
        unique_tokens.update(tokens)
    return sorted(unique_tokens)

In [ ]:
def create_freq_matrix(sentences, unique_tokens, matrix_type, normalization_type):

    freq_matrix = []
    if matrix_type == 'Binary':
        for sentence in sentences:

            tokens = [word.lower() for word in sentence.split() if word.isalnum()]
            
            matrix_row = [1 if token in tokens else 0 for token in unique_tokens]
            freq_matrix.append(matrix_row)
    else: # Occurence
        for sentence in sentences:

            tokens = [word.lower() for word in sentence.split() if word.isalnum()]

            matrix_row = [tokens.count(token) for token in unique_tokens]
            freq_matrix.append(matrix_row)

    # Appliquer la normalisation
    if normalization_type == "Probability":
        for row in freq_matrix:
            row_sum = sum(row)
            for i in range(len(row)):
                row[i] = row[i] / row_sum if row_sum != 0 else 0
    elif normalization_type == "L2":
        for row in freq_matrix:
            row_sum = sum(x**2 for x in row) ** 0.5
            for i in range(len(row)):
                row[i] = row[i] / row_sum if row_sum != 0 else 0

    return freq_matrix

In [7]:
def createSimilarityMatrix(sentences, bow_matrix, distance):
    
    distance_matrix = [[0] * len(sentences) for _ in range(len(sentences))]

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                distance_matrix[i][j] = distance(bow_matrix[i], bow_matrix[j])

    return distance_matrix

In [ ]:
import numpy as np

documents = ["the cat sat on the mat",    "the dog sat on the log",    "cats and dogs are great pets"]




print(np.array(createSimilarityMatrix(documents[:3], tf_idf_bin, distance_euclidean)))
print(np.array(createSimilarityMatrix(documents[:3], tf_idf_occ, distance_euclidean)))

# Descripteurs:

1. Bag of Words

In [30]:
def bow_bin(document, vocabulaire):
    bow_bin = [1 if token in document.split() else 0 for token in vocabulaire]
    return bow_bin

In [31]:
def bow_occ(document, vocabulaire):
    bow_occ = [document.split().count(token) for token in vocabulaire]
    return bow_occ

In [32]:
def bow_N1(bow_occ):
    somme = sum(bow_occ)
    bow = [i/somme for i in bow_occ]
    return bow

In [33]:
def bow_N2(bow_occ):
    somme = 0
    for elt in bow_occ:
        somme += elt**2
    bow = [i/math.sqrt(somme) for i in bow_occ]
    return bow

2. TF-IDF

In [34]:
def TF_bin(document, vocabulaire):
    return [1 if token in document.split() else 0 for token in vocabulaire]

In [35]:
def TF_occ(document, vocabulaire):
    return [document.split().count(token) for token in vocabulaire]

In [36]:
def TF_occ_N(document, vocabulaire):
    """Calcul du TF normalisé (occurrences / nombre de mots) pour chaque terme du vocabulaire dans le document."""
    words = document.split()
    total_words = len(words)
    return [words.count(token) / total_words if total_words > 0 else 0 for token in vocabulaire]

In [37]:
def compute_idf(documents, vocabulaire):
    """Calcul de l'IDF pour chaque terme du vocabulaire."""
    N = len(documents)  # Nombre total de documents
    idf_list = []

    for token in vocabulaire:
        # Compter combien de documents contiennent le terme
        df = sum(1 for doc in documents if token in doc.split())
        if df != 0:
            idf = math.log(N / df)  # Ajout de 1 pour éviter division par zéro
        else:
            idf = 0
        idf_list.append(idf)
    
    return idf_list

def compute_tf_idf(documents, vocabulaire, tf_function):
    """Construction de la matrice TF-IDF."""
    # calcul de l'IDF pour le vocabulaire
    idf_list = compute_idf(documents, vocabulaire)
    
    # construction de la matrice TF-IDF pour chaque document
    tf_idf_matrix = []
    for document in documents:
        # calcul du TF 
        tf_vector = tf_function(document, vocabulaire)
        
        # calcul de TF * IDF
        tf_idf_vector = [tf * idf for tf, idf in zip(tf_vector, idf_list)]
        tf_idf_matrix.append(tf_idf_vector)
    
    return tf_idf_matrix

Distances :

In [8]:
def distance_euclidean(vect1, vect2):
    s = 0
    for i in range(len(vect1)):
        s += math.pow(vect1[i] - vect2[i], 2)
    
    return math.sqrt(s)

In [9]:
def distance_manhattan(vect1, vect2):
    s = 0
    for i in range(len(vect1)):
        s += abs(vect1[i] - vect2[i])
    
    return s

In [10]:
def distance_cosinus(vect1, vect2):
    somme_norm1 = 0
    somme_norm2 = 0
    produit_scalaire = 0
    for i in range(len(vect1)):
        somme_norm1 += vect1[i]**2
        somme_norm2 += vect2[i]**2
        produit_scalaire += vect1[i]*vect2[i]

    return (1 - (produit_scalaire/(math.sqrt(somme_norm1)*math.sqrt(somme_norm2))))

In [11]:
def distance_jaccard(vect1, vect2):
    somme_union = 0
    somme_intersec = 0
    for i in range(len(vect1)):
        if vect1[i] != vect2[i]:
            somme_union += 1
        elif vect1[i] != 0 or vect2[i] != 0: # s'assurer que les deux mots se trouvent au moins une fois dans les documents
            somme_intersec += 1
            
    return (1 - (somme_intersec/somme_union))

In [12]:
def distance_hamming(vect1, vect2):
    somme_diff = 0
    for i in range(len(vect1)):
        if vect1[i] != vect2[i]:
            somme_diff += 1
            
    return somme_diff

In [13]:
def distance_bray_curtis(vect1, vect2):
    somme_abs = 0
    somme = 0
    for i in range(len(vect1)):
        somme_abs += abs(vect1[i] - vect2[i])
        somme += vect1[i] + vect2[i]

    if somme != 0:
        return somme_abs / somme
    else:
        return 0

In [14]:
def distance_kullback_leibler(vect1, vect2):
    somme = 0
    for i in range(len(vect1)):
        if vect1[i] > 0 and vect2[i] > 0:
            somme += vect1[i] * math.log(vect1[i]/vect2[i])

    return somme

Création de la matrice BoW et la matrice de similarité :

In [45]:
import numpy as np # pour l'affichage seulement

def createBoWMatrix(function1, sentences, tokens, function2):

    #matrix = []
    #for sentence in sentences:
    #    if function1 and function2:
    #        matrix.append(function2(function1(sentence, tokens)))
    #    elif function1:
    #        matrix.append(function1(sentence, tokens))
    #    else:
    #        matrix.append(bow_occ(sentence, tokens)
        
    matrix = [function2(function1(sentence, tokens)) if function1 and function2
            else function1(sentence, tokens) if function1
            else bow_occ(sentence, tokens)
            for sentence in sentences]
    
    print("bow_matrix:\n",np.array(matrix), "\n")

    #print(pd.DataFrame(matrix, columns=tokens, index=[f"Phrase_{i+1}" for i in range(len(sentences))]))
    
    return matrix

def createSimilarityMatrix(sentences, bow_matrix, distance):
    
    distance_matrix = [[0] * len(sentences) for _ in range(len(sentences))]

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                distance_matrix[i][j] = distance(bow_matrix[i], bow_matrix[j])

    return distance_matrix
                
    #return euclidean_matrix, manhattan_matrix, cosinus_matrix

- TF-IDF

In [46]:
# Exemple
documents = ["the cat sat on the mat",    "the dog sat on the log",    "cats and dogs are great pets"]

vocabulaire = list(set(" ".join(documents).split()))

# TF-bin
tf_idf_bin = compute_tf_idf(documents, vocabulaire, TF_bin)
print("Matrice TF-IDF avec TF binaire:")
for i, vec in enumerate(tf_idf_bin):
    print(f"Document {i+1}: {vec}")

# TF-occ
tf_idf_occ = compute_tf_idf(documents, vocabulaire, TF_occ)
print("\nMatrice TF-IDF avec TF par occurrences:")
for i, vec in enumerate(tf_idf_occ):
    print(f"Document {i+1}: {vec}")

# TF-occ-N
tf_idf_occ_N = compute_tf_idf(documents, vocabulaire, TF_occ_N)
print("\nMatrice TF-IDF avec TF par occurrences normalisees:")
for i, vec in enumerate(tf_idf_occ_N):
    print(f"Document {i+1}: {vec}")



Matrice TF-IDF avec TF binaire:
Document 1: [0.4054651081081644, 0.0, 0.0, 1.0986122886681098, 0.0, 0.0, 0.4054651081081644, 1.0986122886681098, 0.4054651081081644, 0.0, 0.0, 0.0, 0.0]
Document 2: [0.4054651081081644, 0.0, 0.0, 0.0, 0.0, 1.0986122886681098, 0.4054651081081644, 0.0, 0.4054651081081644, 0.0, 0.0, 0.0, 1.0986122886681098]
Document 3: [0.0, 1.0986122886681098, 1.0986122886681098, 0.0, 1.0986122886681098, 0.0, 0.0, 0.0, 0.0, 1.0986122886681098, 1.0986122886681098, 1.0986122886681098, 0.0]

Matrice TF-IDF avec TF par occurrences:
Document 1: [0.4054651081081644, 0.0, 0.0, 1.0986122886681098, 0.0, 0.0, 0.8109302162163288, 1.0986122886681098, 0.4054651081081644, 0.0, 0.0, 0.0, 0.0]
Document 2: [0.4054651081081644, 0.0, 0.0, 0.0, 0.0, 1.0986122886681098, 0.8109302162163288, 0.0, 0.4054651081081644, 0.0, 0.0, 0.0, 1.0986122886681098]
Document 3: [0.0, 1.0986122886681098, 1.0986122886681098, 0.0, 1.0986122886681098, 0.0, 0.0, 0.0, 0.0, 1.0986122886681098, 1.0986122886681098, 1.09

In [47]:
import numpy as np
print(np.array(createSimilarityMatrix(documents[:3], tf_idf_bin, distance_euclidean)))
print(np.array(createSimilarityMatrix(documents[:3], tf_idf_occ, distance_euclidean)))
print(np.array(createSimilarityMatrix(documents[:3], tf_idf_occ_N, distance_euclidean)))
print("\n------------------------------------------------")
print(np.array(createSimilarityMatrix(documents[:3], tf_idf_bin, distance_manhattan)))
print(np.array(createSimilarityMatrix(documents[:3], tf_idf_occ, distance_manhattan)))
print(np.array(createSimilarityMatrix(documents[:3], tf_idf_occ_N, distance_manhattan)))
print("\n------------------------------------------------")
print(np.array(createSimilarityMatrix(documents[:3], tf_idf_bin, distance_cosinus)))
print(np.array(createSimilarityMatrix(documents[:3], tf_idf_occ, distance_cosinus)))
print(np.array(createSimilarityMatrix(documents[:3], tf_idf_occ_N, distance_cosinus)))
print("\n------------------------------------------------")
print(np.array(createSimilarityMatrix(documents[:3], tf_idf_bin, distance_bray_curtis)))
print(np.array(createSimilarityMatrix(documents[:3], tf_idf_occ, distance_bray_curtis)))
print(np.array(createSimilarityMatrix(documents[:3], tf_idf_occ_N, distance_bray_curtis)))



[[0.         2.19722458 3.18571774]
 [2.19722458 0.         3.18571774]
 [3.18571774 3.18571774 0.        ]]
[[0.         2.19722458 3.26220836]
 [2.19722458 0.         3.26220836]
 [3.26220836 3.26220836 0.        ]]
[[0.         0.3662041  0.54370139]
 [0.3662041  0.         0.54370139]
 [0.54370139 0.54370139 0.        ]]

------------------------------------------------
[[ 0.          4.39444915 10.00529363]
 [ 4.39444915  0.         10.00529363]
 [10.00529363 10.00529363  0.        ]]
[[ 0.          4.39444915 10.41075874]
 [ 4.39444915  0.         10.41075874]
 [10.41075874 10.41075874  0.        ]]
[[0.         0.73240819 1.73512646]
 [0.73240819 0.         1.73512646]
 [1.73512646 1.73512646 0.        ]]

------------------------------------------------
[[0.        0.8303446 1.       ]
 [0.8303446 0.        1.       ]
 [1.        1.        0.       ]]
[[0.         0.70990532 1.        ]
 [0.70990532 0.         1.        ]
 [1.         1.         0.        ]]
[[0.         0.7099

------------------------------------------------------- (bow part) ------------------------------------------------------
# deroulement :
- create bow matrix (avec les fonctions, tokens, phrases)
- create similarity matrix (avec la fonction, bow, phrases)

Exemple (Corpus chat-chien)

In [95]:
# mettre en commentaire les deux lignes suivantes pour executer le corpus de Chirac
#sentences = ["the cat sat on the mat",    "the dog sat on the log",    "cats and dogs are great pets"] ###
#tokens = list(set(" ".join(sentences).split())) ###


sentences_prime = []
sentences_deux_primes = []
for sent in sentences:
    sentences_prime.append((sent + " ")*2)
    sentences_deux_primes.append((sent + " ")*4)

print(sentences_prime)
print(sentences_deux_primes)

["la confiance que vous venez de me témoigner, je veux y répondre en m'engageant dans l'action avec détermination la confiance que vous venez de me témoigner, je veux y répondre en m'engageant dans l'action avec détermination ", "mes chers compatriotes de métropole, d'outre-mer et de l'étranger,\nnous venons de vivre un temps de grave inquiétude pour la nation mes chers compatriotes de métropole, d'outre-mer et de l'étranger,\nnous venons de vivre un temps de grave inquiétude pour la nation ", 'mais ce soir, dans un grand élan la france a réaffirmé son attachement aux valeurs de la république mais ce soir, dans un grand élan la france a réaffirmé son attachement aux valeurs de la république ', 'je salue la france, fidèle à elle-même, fidèle à ses grands idéaux, fidèle à sa vocation universelle et humaniste je salue la france, fidèle à elle-même, fidèle à ses grands idéaux, fidèle à sa vocation universelle et humaniste ', "je salue la france qui, comme toujours dans les moments difficil

** test **

1. Résultats des phrases (normales)

- Distance Curtis :
  Chats & Chiens : bons resultats
  Chirac :

In [97]:
print("Phrases :\n", sentences[:3])

print("\n---------------------------- Bin Jaccard ----------------------------\n")
print(np.array(createSimilarityMatrix(sentences[:3], createBoWMatrix(bow_bin, sentences[:3], tokens, None), distance_bray_curtis)))

print("\n---------------------------- Bin Hamming ----------------------------\n")
print(np.array(createSimilarityMatrix(sentences[:3], createBoWMatrix(bow_occ, sentences[:3], tokens, None), distance_bray_curtis)))

print("\n---------------------------- N1 ----------------------------\n")
print(np.array(createSimilarityMatrix(sentences[:3], createBoWMatrix(bow_occ, sentences[:3], tokens, bow_N1), distance_bray_curtis)))

print("\n---------------------------- N2 ----------------------------\n")
print(np.array(createSimilarityMatrix(sentences[:3], createBoWMatrix(bow_occ, sentences[:3], tokens, bow_N2), distance_bray_curtis)))


Phrases :
 ["la confiance que vous venez de me témoigner, je veux y répondre en m'engageant dans l'action avec détermination", "mes chers compatriotes de métropole, d'outre-mer et de l'étranger,\nnous venons de vivre un temps de grave inquiétude pour la nation", 'mais ce soir, dans un grand élan la france a réaffirmé son attachement aux valeurs de la république']

---------------------------- Bin Jaccard ----------------------------

bow_matrix:
 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

2. Résultats des phrases'

In [30]:
print("Phrases' :\n", sentences_prime[:3])

print("\n---------------------------- Bin ----------------------------\n")
createSimilarityMatrix(sentences_prime[:3], createBoWMatrix(bow_bin, sentences_prime[:3], tokens, None)) 

print("\n---------------------------- Occ ----------------------------\n")
createSimilarityMatrix(sentences_prime[:3], createBoWMatrix(bow_occ, sentences_prime[:3], tokens, None)) 

print("\n---------------------------- N1 ----------------------------\n")

createSimilarityMatrix(sentences_prime[:3], createBoWMatrix(bow_occ, sentences_prime[:3], tokens, bow_N1)) # N1

print("\n---------------------------- N2 ----------------------------\n")

createSimilarityMatrix(sentences_prime[:3], createBoWMatrix(bow_occ, sentences_prime[:3], tokens, bow_N2)) # N2

Phrases' :
 ['the cat sat on the mat the cat sat on the mat ', 'the dog sat on the log the dog sat on the log ', 'cats and dogs are great pets cats and dogs are great pets ']

---------------------------- Bin ----------------------------

bow_matrix:
 [[1 0 1 1 0 0 0 1 0 1 0 0 0]
 [0 0 0 1 1 0 0 1 1 1 0 0 0]
 [0 1 0 0 0 1 1 0 0 0 1 1 1]] 



TypeError: createSimilarityMatrix() missing 1 required positional argument: 'distance'

3. Résultats des phrases"

In [19]:
print("Phrases'' :\n", sentences_deux_primes[:3])

print("\n---------------------------- Bin ----------------------------\n")
createSimilarityMatrix(sentences_deux_primes[:3], createBoWMatrix(bow_bin, sentences_deux_primes[:3], tokens, None)) 

print("\n---------------------------- Occ ----------------------------\n")
createSimilarityMatrix(sentences_deux_primes[:3], createBoWMatrix(bow_occ, sentences_deux_primes[:3], tokens, None)) 

print("\n---------------------------- N1 ----------------------------\n")

createSimilarityMatrix(sentences_deux_primes[:3], createBoWMatrix(bow_occ, sentences_prime[:3], tokens, bow_N1)) # N1

print("\n---------------------------- N2 ----------------------------\n")

createSimilarityMatrix(sentences_deux_primes[:3], createBoWMatrix(bow_occ, sentences_prime[:3], tokens, bow_N2)) # N2

Phrases'' :
 ['the cat sat on the mat the cat sat on the mat the cat sat on the mat the cat sat on the mat ', 'the dog sat on the log the dog sat on the log the dog sat on the log the dog sat on the log ', 'cats and dogs are great pets cats and dogs are great pets cats and dogs are great pets cats and dogs are great pets ']

---------------------------- Bin ----------------------------


Matrice de similarité avec la distance euclidéene:
[[0.         2.         3.31662479]
 [2.         0.         3.31662479]
 [3.31662479 3.31662479 0.        ]]

Matrice de similarité avec la distance manhattan:
[[ 0  4 11]
 [ 4  0 11]
 [11 11  0]]

Matrice de similarité avec la distance cosinus:
[[0.  0.4 1. ]
 [0.4 0.  1. ]
 [1.  1.  0. ]]

---------------------------- Occ ----------------------------


Matrice de similarité avec la distance euclidéene:
[[ 0.          8.         14.96662955]
 [ 8.          0.         14.96662955]
 [14.96662955 14.96662955  0.        ]]

Matrice de similarité avec la d

# Brouillon

In [3]:
def TF_IDF_occ_total(token, documents):
    occ_total = 0
    tf = []
    for token in tokens:
        idf = 0
        tf_token = []
        for document in documents:
            occ = document.count(token)
            tf_token.append(occ)
            if occ:
                idf += 1
        occ_total += sum(tf_token)
        tf_token.append(idf)
    
    for token in range(len(tokens)-1):
        
    return (

In [57]:
import pandas as pd

def bow_bin(documents, vocabulaire):
    # Créer une liste pour stocker chaque vecteur binaire
    bow_matrix = []

    # Parcourir chaque document et créer le vecteur binaire
    for document in documents:
        tokens_document = document.split()
        bow_bin = [1 if token in tokens_document else 0 for token in vocabulaire]
        bow_matrix.append(bow_bin)

    # Créer un DataFrame avec les documents en lignes et les tokens en colonnes
    df = pd.DataFrame(bow_matrix, columns=vocabulaire, index=[f"Doc_{i+1}" for i in range(len(documents))])

    return df

# Exemple d'utilisation
documents = ["hello world this is a test", "hello again world", "python is fun"]
vocabulaire = ["hello", "test", "python", "world", "is", "fun"]

df = bow_bin(documents, vocabulaire)

print(df)


       hello  test  python  world  is  fun
Doc_1      1     1       0      1   1    0
Doc_2      1     0       0      1   0    0
Doc_3      0     0       1      0   1    1


In [13]:
import numpy as np # pour l'affichage seulement

def createBoWMatrix(function1, sentences, tokens, function2):

    #matrix = []
    #for sentence in sentences:
    #    if function1 and function2:
    #        matrix.append(function2(function1(sentence, tokens)))
    #    elif function1:
    #        matrix.append(function1(sentence, tokens))
    #    else:
    #        matrix.append(bow_occ(sentence, tokens)
        
    matrix = [function2(function1(sentence, tokens)) if function1 and function2
            else function1(sentence, tokens) if function1
            else bow_occ(sentence, tokens)
            for sentence in sentences ]
    
    return matrix

def createSimilarityMatrix(sentences, bow_matrix):
    
    euclidean_matrix = [[0] * len(sentences) for _ in range(len(sentences))]
    manhattan_matrix = [[0] * len(sentences) for _ in range(len(sentences))]
    cosinus_matrix = [[0] * len(sentences) for _ in range(len(sentences))]

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                euclidean_matrix[i][j] = distance_euclidean(bow_matrix[i], bow_matrix[j])
                manhattan_matrix[i][j] = distance_manhattan(bow_matrix[i], bow_matrix[j])
                cosinus_matrix[i][j] = distance_cosinus(bow_matrix[i], bow_matrix[j])

    print("\nMatrice de similarité avec la distance euclidéene:")
    print(np.array(euclidean_matrix))
    
    print("\nMatrice de similarité avec la distance manhattan:")
    print(np.array(manhattan_matrix))
    
    print("\nMatrice de similarité avec la distance cosinus:")
    print(np.array(cosinus_matrix))
                
    #return euclidean_matrix, manhattan_matrix, cosinus_matrix